# ~Simple~ Multiple Linear Regression

- Review Simple Linear Regression
    - Correlation
    - `statsmodels.formula.api as smf`
        - Begin examples with "amazonbooks.csv" (`ab_noNaN`)
    - `import statsmodels.api as sm`
        - Estimated ***estimated*** ("intercept" and "slope") ***coefficients*** and ***fitted values*** $\hat y_i = \hat \beta_0 + \hat \beta_1x_i$ 
        - ***Statistical inference*** using a ***null hypothesis*** for ***model parameters*** $H_0: \beta_k$ and ***estimated coefficient*** $\hat \beta_k$ (***test statistic***)
        
        
- Definitions
    - $y_i$ (outcomes, responses, dependent/endogenous variables)
    - $x_i$ (covariates, features, independent/exogenous/explanatory/predictor variables)
    - ***fitted values / predictions*** $\hat y_i = \hat \beta_0 + \hat \beta_1x_i$ 
        - ***residuals*** $e_i = \hat \epsilon_i = y_i - \hat y_i \neq \epsilon_i$
            - ***proportion of variation explained*** $R^2 = r_{y\hat y}^2 = 1-\frac{\sum_{i=1}^n(y_i-\hat y_i)^2}{\sum_{i=1}^n(y_i-\bar y_i)^2} = 1-\frac{\frac{1}{n-1}\sum_{i=1}^ne_i^2}{s_y^2}$
    - ***extrapolation predictions*** $\tilde y_i = \hat \beta_0 + \hat \beta_1 \tilde x_i$
    
    
- Multiple Linear Regression
    - Indicator Variables
    - Interactions 
    - Model building
    - Train-Test Framework
        - "In Sample" versus "Out of Sample" performance



## "Review" Simple Linear Regression

In [1]:
import pandas as pd
ab = pd.read_csv("../../amazonbooks.csv", encoding="ISO-8859-1")#.dropna()
#print(ab.shape)
#ab.isnull().sum()
ab_noNaN = ab.drop(['Weight_oz','Width','Height'], axis=1).dropna()
ab_noNaN['Pub year'] = ab_noNaN['Pub year'].astype(int)
ab_noNaN['NumPages'] = ab_noNaN['NumPages'].astype(int)
ab_noNaN['Hard_or_Paper'] = ab_noNaN['Hard_or_Paper'].astype("category")
#print(ab_noNaN.shape)
#ab_noNaN.dtypes
ab_noNaN

,Title,Author,List Price,Amazon Price,Hard_or_Paper,NumPages,Publisher,Pub year,ISBN-10,Thick
0,"1,001 Facts that Will Scare the S#*t Out of Yo...",Cary McNeal,12.95,5.18,P,304,Adams Media,2010,1605506249,0.8
1,21: Bringing Down the House - Movie Tie-In: Th...,Ben Mezrich,15.00,10.20,P,273,Free Press,2008,1416564195,0.7
2,100 Best-Loved Poems (Dover Thrift Editions),Smith,1.50,1.50,P,96,Dover Publications,1995,486285537,0.3
3,1421: The Year China Discovered America,Gavin Menzies,15.99,10.87,P,672,Harper Perennial,2008,61564893,1.6
4,1493: Uncovering the New World Columbus Created,Charles C. Mann,30.50,16.77,P,720,Knopf,2011,307265722,1.4
...,...,...,...,...,...,...,...,...,...,...
320,Where the Sidewalk Ends,Shel Silverstein,18.99,12.24,H,192,HarperCollins,2004,60572345,1.1
321,White Privilege,Paula S. Rothenberg,27.55,27.55,P,160,Worth Publishers,2011,1429233443,0.7
322,Why I wore lipstick,Geralyn Lucas,12.95,5.18,P,224,St Martin's Griffin,2005,031233446X,0.7
323,"Worlds Together, Worlds Apart: A History of th...",Robert Tignor,97.50,97.50,P,480,W. W. Norton & Company,2010,393934942,0.9


In [ ]:
import plotly.offline as pyo; pyo.init_notebook_mode()

In [3]:
import plotly.express as px
px.scatter(ab_noNaN, x='List Price', y='Amazon Price', trendline="ols")


# ***correlation*** $r_{xy} = \frac{\text{Cov}(x,y)}{s_xs_y} \in [-1,1]$

```python
np.cov(x,y, ddof=1)[0,1]/(x.std(ddof=1)*y.std(ddof=1)) # np.corrcoef(x,y)[0,1]
# np.cov(x,y, ddof=1), x.var(ddof=1), y.var(ddof=1)
```


## $\require{cancel}\sqrt{r^2} = r_{xy} = \frac{\displaystyle \sum_{i=0}^{n-1\color{gray}{[=318]}} (x_i - \bar x)(y_i - \bar y)}{s_xs_y (n-1)} = \frac{\displaystyle \sum_{i=0}^{n-1\color{gray}{[=318]}} (x_i - \bar x)(y_i - \bar y)}{\sqrt{\frac{\sum_{i=0}^{n-1} (x_i - \bar x)^2}{\cancel{n-1}}} \sqrt{\frac{\sum_{i=0}^{n-1} (y_i - \bar y)^2}{\cancel{n-1}}} \cancel{(n-1)}}$

And we explored how this was related to $\sigma$ in the theortical model 

# $y_i = \beta_0+\beta_1x_i + \epsilon_i, \quad  \epsilon_i \sim \mathcal N (0, \sigma^2)$


![](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d4/Correlation_examples2.svg/2880px-Correlation_examples2.svg.png)



# Then we fit the model $\hat y = \hat \beta_0+\hat \beta_1x$

$$ \huge 
\begin{aligned}
\hat \beta_1  &={} r_{xy} \frac {s_{y}}{s_{x}} \quad \text{ slope coefficent}\\
 &={} {\frac {\sum _{i=1}^{n}(x_{i}-{\bar {x}})(y_{i}-{\bar {y}})}{\sum _{i=1}^{n}(x_{i}-{\bar {x}})^{2}}}\\\\
\hat \beta_0  &={} \bar{y} -  \bar{x}\beta_1 \quad \text{ intercept coefficient}
\end{aligned}
$$

In [4]:
x = ab_noNaN["List Price"]
y = ab_noNaN["Amazon Price"]

import numpy as np
beta1_hat = np.corrcoef(x,y)[0,1] * (y.std()/x.std())
beta0_hat = y.mean() - beta1_hat*x.mean()
beta0_hat, beta1_hat

(-2.6675542874308995, 0.8500031270650632)

In [6]:
# import statsmodels.api as sm
import statsmodels.formula.api as smf

simple_linear_regression_model = smf.ols('Q("Amazon Price") ~ Q("List Price")', data=ab_noNaN)
model_fit = simple_linear_regression_model.fit() # creates line of best fit
model_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      Q("Amazon Price")   R-squared:                       0.913
Model:                            OLS   Adj. R-squared:                  0.912
Method:                 Least Squares   F-statistic:                     3307.
Date:                Mon, 20 Nov 2023   Prob (F-statistic):          9.09e-170
Time:                        13:33:02   Log-Likelihood:                -867.62
No. Observations:                 319   AIC:                             1739.
Df Residuals:                     317   BIC:                             1747.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -2.6676      0.341     -7.825      0.000      -3.338      -1.997
Q("List Price")     0.8500      0.015     57.506      0.000       0.821       0.879
==============================================================================
Omnibus:                      104.183   Durbin-Watson:                   2.015
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1018.441
Skew:                           1.036   Prob(JB):                    7.05e-222
Kurtosis:                      11.505   Cond. No.                         38.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Hypothesis testing the association between x and y

# $H_0: \beta_1= 0 \quad x \text{ and } y \text{ have no linear association}$
- for the model $y_i = \beta_0 + \beta_1 x_i + \epsilon_i, \epsilon_i \sim \mathcal N (0, \sigma)$

# $H_1: \beta_1 \neq 0 \quad x \text{ and } y \text{ have a linear association}$


The "slope" coefficient $\hat \beta_1$ is our test statistic and it has an implied sampling distribution under $H_0$ from which we can calculate a ***p-value***


# Let's restate the above more carefully!

## $H_0: \beta_1= 0 \quad \text{There is no detectable average change in $y$ for a unit increase in $x$}$
- for the model $y_i = \beta_0 + \beta_1 x_i + \epsilon_i, \epsilon_i \sim \mathcal N (0, \sigma)$

## $H_1: \text{ $H_0$ is false}$



In [9]:
from scipy import stats
n = 100 
x = stats.uniform(loc=0, scale=10).rvs(n)
beta0 = 10
beta1 = 2
error = stats.norm(loc = 0, scale = 10).rvs(n) # noise ... we considerd 1 and 4 for the std(scale)
y_with_noise = beta0 + beta1*x*x + error # expected_value_of_y = beta0 + beta1*x
#y_with_noise = y_with_noise*2
df = pd.DataFrame({'x':x, 'y':y_with_noise})
px.scatter(df, x='x', y='y', trendline="ols")

In [10]:
simple_linear_regression_model = smf.ols('y ~ x', data=df)
model_fit = simple_linear_regression_model.fit() # creates line of best fit
model_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.909
Model:                            OLS   Adj. R-squared:                  0.908
Method:                 Least Squares   F-statistic:                     978.5
Date:                Mon, 20 Nov 2023   Prob (F-statistic):           8.45e-53
Time:                        13:38:59   Log-Likelihood:                -428.54
No. Observations:                 100   AIC:                             861.1
Df Residuals:                      98   BIC:                             866.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -20.6732      3.564     -5.801      0.000     -27.745     -13.601
x             19.2942      0.617     31.281      0.000      18.070      20.518
==============================================================================
Omnibus:                        6.809   Durbin-Watson:                   1.772
Prob(Omnibus):                  0.033   Jarque-Bera (JB):                6.689
Skew:                           0.632   Prob(JB):                       0.0353
Kurtosis:                       3.079   Cond. No.                         11.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
E[y] = beta0 + beat1*x
E[y] = 0 + beat1*x

In [14]:
# simple_linear_regression_model = smf.ols('y ~ x', data=df)
# that is the same as (that actually means)
# simple_linear_regression_model = smf.ols('y ~ 1 + x', data=df)
# simple_linear_regression_model = smf.ols('y ~ -1 + x', data=df)

import statsmodels.api as sm

ab_noNaN

x = ab_noNaN["List Price"]
X = sm.add_constant(x)
X # design matrix

,const,List Price
0,1.0,12.95
1,1.0,15.00
2,1.0,1.50
3,1.0,15.99
4,1.0,30.50
...,...,...
320,1.0,18.99
321,1.0,27.55
322,1.0,12.95
323,1.0,97.50


In [13]:
x

0      12.95
1      15.00
2       1.50
3      15.99
4      30.50
       ...  
320    18.99
321    27.55
322    12.95
323    97.50
324    16.99
Name: List Price, Length: 319, dtype: float64

In [16]:
simple_linear_regression_model = smf.ols('Q("Amazon Price") ~ Q("List Price")', data=ab_noNaN)
model_fit = simple_linear_regression_model.fit() # creates line of best fit
model_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      Q("Amazon Price")   R-squared:                       0.913
Model:                            OLS   Adj. R-squared:                  0.912
Method:                 Least Squares   F-statistic:                     3307.
Date:                Mon, 20 Nov 2023   Prob (F-statistic):          9.09e-170
Time:                        14:31:03   Log-Likelihood:                -867.62
No. Observations:                 319   AIC:                             1739.
Df Residuals:                     317   BIC:                             1747.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -2.6676      0.341     -7.825      0.000      -3.338      -1.997
Q("List Price")     0.8500      0.015     57.506      0.000       0.821       0.879
==============================================================================
Omnibus:                      104.183   Durbin-Watson:                   2.015
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1018.441
Skew:                           1.036   Prob(JB):                    7.05e-222
Kurtosis:                      11.505   Cond. No.                         38.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [17]:
y = ab_noNaN["Amazon Price"]
simple_linear_regression_model_smstyle = sm.OLS(y, X)
res = simple_linear_regression_model_smstyle.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           Amazon Price   R-squared:                       0.913
Model:                            OLS   Adj. R-squared:                  0.912
Method:                 Least Squares   F-statistic:                     3307.
Date:                Mon, 20 Nov 2023   Prob (F-statistic):          9.09e-170
Time:                        14:32:28   Log-Likelihood:                -867.62
No. Observations:                 319   AIC:                             1739.
Df Residuals:                     317   BIC:                             1747.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.6676      0.341     -7.825      0.000      -3.338      -1.997
List Price     0.8500      0.015     57.506      0.000       0.821       0.879
==============================================================================
Omnibus:                      104.183   Durbin-Watson:                   2.015
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1018.441
Skew:                           1.036   Prob(JB):                    7.05e-222
Kurtosis:                      11.505   Cond. No.                         38.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [68]:
from scipy import stats
n = 100 
x = stats.uniform(loc=0, scale=10).rvs(n)
beta0 = 10
beta1 = 2
error = stats.norm(loc = 0, scale = 10).rvs(n) # noise ... we considerd 1 and 4 for the std(scale)
y_with_noise = beta0 + beta1*x*x + error # expected_value_of_y = beta0 + beta1*x
#y_with_noise = y_with_noise*2
df = pd.DataFrame({'x':x, 'y':y_with_noise})
fig = px.scatter(df, x='x', y='y', trendline="ols")

# added after for fun
import plotly.graph_objects as go
fig.add_trace(go.Scatter(x=model_fit.model.exog[:,1], 
                         y=model_fit.predict(), mode="markers"))


In [24]:
df

,x,y
0,8.258911,156.906116
1,5.921928,102.784122
2,4.699145,53.306531
3,3.950175,51.840289
4,1.515485,28.763701
...,...,...
95,5.102888,60.129992
96,1.338488,-1.777654
97,7.605927,102.751693
98,9.391027,202.194940


In [61]:
df['x_copy'] = df.x
df

,x,y,x_copy
0,3.562297,47.353769,3.562297
1,7.505871,136.903353,7.505871
2,7.178106,104.358780,7.178106
3,7.187630,123.084991,7.187630
4,6.206050,81.441196,6.206050
...,...,...,...
95,7.381110,111.563209,7.381110
96,9.216991,171.815448,9.216991
97,6.716886,92.591602,6.716886
98,0.980912,18.372516,0.980912


In [ ]:
# added after
# E[y] = beta0 + beta1 x + beta2 * x*x

In [ ]:
# added after 
# E[y] = beta0 + beta1 x + beta1^2 * z

In [62]:
simple_linear_regression_model = smf.ols('y ~ x + x:x_copy', data=df)
model_fit = simple_linear_regression_model.fit() # creates line of best fit
model_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.968
Model:                            OLS   Adj. R-squared:                  0.967
Method:                 Least Squares   F-statistic:                     1460.
Date:                Mon, 20 Nov 2023   Prob (F-statistic):           4.01e-73
Time:                        15:21:45   Log-Likelihood:                -362.50
No. Observations:                 100   AIC:                             731.0
Df Residuals:                      97   BIC:                             738.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     12.0382      3.195      3.768      0.000       5.698      18.379
x             -0.5482      1.429     -0.384      0.702      -3.385       2.289
x:x_copy       2.0264      0.142     14.225      0.000       1.744       2.309
==============================================================================
Omnibus:                        1.235   Durbin-Watson:                   1.917
Prob(Omnibus):                  0.539   Jarque-Bera (JB):                1.112
Skew:                          -0.076   Prob(JB):                        0.574
Kurtosis:                       2.506   Cond. No.                         152.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [39]:
df.y
x = df.x
X = sm.add_constant(x)
X['x^2'] = df.x*df.x
X

,const,x,x^2
0,1.0,8.258911,68.209618
1,1.0,5.921928,35.069226
2,1.0,4.699145,22.081964
3,1.0,3.950175,15.603885
4,1.0,1.515485,2.296695
...,...,...,...
95,1.0,5.102888,26.039467
96,1.0,1.338488,1.791549
97,1.0,7.605927,57.850124
98,1.0,9.391027,88.191387


In [40]:
simple_linear_regression_model_smstyle = sm.OLS(df.y, X)
res = simple_linear_regression_model_smstyle.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.967
Model:                            OLS   Adj. R-squared:                  0.966
Method:                 Least Squares   F-statistic:                     1412.
Date:                Mon, 20 Nov 2023   Prob (F-statistic):           1.91e-72
Time:                        14:42:57   Log-Likelihood:                -381.48
No. Observations:                 100   AIC:                             769.0
Df Residuals:                      97   BIC:                             776.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.4606      4.235      1.526      0.130      -1.944      14.865
x              1.9450      1.783      1.091      0.278      -1.595       5.485
x^2            1.8359      0.163     11.248      0.000       1.512       2.160
==============================================================================
Omnibus:                        0.890   Durbin-Watson:                   2.104
Prob(Omnibus):                  0.641   Jarque-Bera (JB):                0.919
Skew:                          -0.218   Prob(JB):                        0.632
Kurtosis:                       2.826   Cond. No.                         193.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [43]:
simple_linear_regression_model_indicator = smf.ols('Q("Amazon Price") ~ Hard_or_Paper', data=ab_noNaN)
model_fit = simple_linear_regression_model_indicator.fit() # creates line of best fit
model_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      Q("Amazon Price")   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.365
Date:                Mon, 20 Nov 2023   Prob (F-statistic):              0.244
Time:                        14:48:35   Log-Likelihood:                -1255.5
No. Observations:                 319   AIC:                             2515.
Df Residuals:                     317   BIC:                             2523.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             14.2794      1.340     10.654      0.000      11.642      16.916
Hard_or_Paper[T.P]    -1.8324      1.568     -1.168      0.244      -4.918       1.253
==============================================================================
Omnibus:                      426.483   Durbin-Watson:                   1.791
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            36201.746
Skew:                           6.439   Prob(JB):                         0.00
Kurtosis:                      53.574   Cond. No.                         3.62
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [44]:
px.box(ab_noNaN, y="Amazon Price", x='Hard_or_Paper')

In [ ]:
E[y] = beta_0 + beta_1 * 1_[Paper](Hard_or_Paper)

In [48]:
x = ab_noNaN["Hard_or_Paper"].copy()
X = sm.add_constant(x)
X['Paper'] = (x=='P').astype(int)
del X["Hard_or_Paper"]
X # design matrix

,const,Paper
0,1.0,1
1,1.0,1
2,1.0,1
3,1.0,1
4,1.0,1
...,...,...
320,1.0,0
321,1.0,1
322,1.0,1
323,1.0,1


In [49]:
y = ab_noNaN["Amazon Price"]
simple_linear_regression_model_smstyle = sm.OLS(y, X)
res = simple_linear_regression_model_smstyle.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           Amazon Price   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.365
Date:                Mon, 20 Nov 2023   Prob (F-statistic):              0.244
Time:                        14:54:59   Log-Likelihood:                -1255.5
No. Observations:                 319   AIC:                             2515.
Df Residuals:                     317   BIC:                             2523.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         14.2794      1.340     10.654      0.000      11.642      16.916
Paper         -1.8324      1.568     -1.168      0.244      -4.918       1.253
==============================================================================
Omnibus:                      426.483   Durbin-Watson:                   1.791
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            36201.746
Skew:                           6.439   Prob(JB):                         0.00
Kurtosis:                      53.574   Cond. No.                         3.62
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [50]:
px.scatter(ab_noNaN, x='List Price', y='Amazon Price', color='Hard_or_Paper',trendline="ols")

In [51]:
simple_linear_regression_model_indicator = \
  smf.ols('Q("Amazon Price") ~ Q("List Price")*Hard_or_Paper', data=ab_noNaN)
model_fit = simple_linear_regression_model_indicator.fit() # creates line of best fit
model_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      Q("Amazon Price")   R-squared:                       0.918
Model:                            OLS   Adj. R-squared:                  0.917
Method:                 Least Squares   F-statistic:                     1173.
Date:                Mon, 20 Nov 2023   Prob (F-statistic):          1.57e-170
Time:                        14:56:39   Log-Likelihood:                -857.63
No. Observations:                 319   AIC:                             1723.
Df Residuals:                     315   BIC:                             1738.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                             -3.8435      0.849     -4.528      0.000      -5.513      -2.174
Hard_or_Paper[T.P]                     1.4748      0.922      1.599      0.111      -0.340       3.289
Q("List Price")                        0.8371      0.035     23.979      0.000       0.768       0.906
Q("List Price"):Hard_or_Paper[T.P]     0.0269      0.038      0.700      0.485      -0.049       0.102
==============================================================================
Omnibus:                      104.718   Durbin-Watson:                   2.043
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1215.942
Skew:                           0.985   Prob(JB):                    9.15e-265
Kurtosis:                      12.360   Cond. No.                         177.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [56]:
x = ab_noNaN[["List Price","Hard_or_Paper"]].copy()
X = sm.add_constant(x)
X['Paper'] = (X["Hard_or_Paper"]=='P').astype(int)
del X["Hard_or_Paper"]
X["List Price times Paper"] = X['List Price']*X['Paper']
X # design matrix

,const,List Price,Paper,List Price times Paper
0,1.0,12.95,1,12.95
1,1.0,15.00,1,15.00
2,1.0,1.50,1,1.50
3,1.0,15.99,1,15.99
4,1.0,30.50,1,30.50
...,...,...,...,...
320,1.0,18.99,0,0.00
321,1.0,27.55,1,27.55
322,1.0,12.95,1,12.95
323,1.0,97.50,1,97.50


In [57]:
y = ab_noNaN["Amazon Price"]
simple_linear_regression_model_smstyle = sm.OLS(y, X)
res = simple_linear_regression_model_smstyle.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           Amazon Price   R-squared:                       0.918
Model:                            OLS   Adj. R-squared:                  0.917
Method:                 Least Squares   F-statistic:                     1173.
Date:                Mon, 20 Nov 2023   Prob (F-statistic):          1.57e-170
Time:                        15:00:10   Log-Likelihood:                -857.63
No. Observations:                 319   AIC:                             1723.
Df Residuals:                     315   BIC:                             1738.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                     -3.8435      0.849     -4.528      0.000      -5.513      -2.174
List Price                 0.8371      0.035     23.979      0.000       0.768       0.906
Paper                      1.4748      0.922      1.599      0.111      -0.340       3.289
List Price times Paper     0.0269      0.038      0.700      0.485      -0.049       0.102
==============================================================================
Omnibus:                      104.718   Durbin-Watson:                   2.043
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1215.942
Skew:                           0.985   Prob(JB):                    9.15e-265
Kurtosis:                      12.360   Cond. No.                         177.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
# add all this stuff below after as part of a question/conversation 
E[y] = beta0 + beta1*ListPrice + 
       beta2*1_[Paper](Hard_or_Paper) + 
       beta3*'List Price times Paper'

$$E[y] = \beta_0 + \beta_1 \times \text{ListPrice} + \beta_2 \times 1_{[\text{'P'}]}(\text{Hard_or_Paper}) + \beta_3 \times \text{ListPrice}\times 1_{[\text{'P'}]}(\text{Hard_or_Paper})$$
- not paperback: $\beta_0 + \beta_1 \times \text{ListPrice}$
- are paperback: $(\beta_0+\beta_2) + (\beta_1+\beta_3) \times \text{ListPrice}$